In [1]:
import secret_keys as key
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()
import pandas as pd

In [2]:
import tweepy

# Creating the authentication object
auth = tweepy.OAuthHandler(key.consumer_key, key.consumer_secret)
# Setting your access token and secret
auth.set_access_token(key.access_token, key.access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth,wait_on_rate_limit=True) 

In [14]:
search_words = '#corona OR #covid19 OR #covid OR #vaccine'
date_since = '2020-03-27'

In [18]:
tweets = tweepy.Cursor(api.search ,
                       q = search_words,
                       lang = 'en',
                       since = date_since
                      ).items(2000)

In [19]:
tweet_list = [[ tweet.geo, tweet.text, tweet.user.screen_name, tweet.user.location ] for tweet in tqdm_notebook(tweets,total = 2000)]

In [20]:
df_tweets = pd.DataFrame(tweet_list, columns = ['geo', 'tweet' , 'user_handle','location'])

In [21]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 4 columns):
geo            2 non-null object
tweet          2000 non-null object
user_handle    2000 non-null object
location       2000 non-null object
dtypes: object(4)
memory usage: 62.6+ KB


In [22]:
# https://towardsdatascience.com/another-twitter-sentiment-analysis-bb5b01ebad90
# Tweet data cleaning
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))
def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
#     lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
#     words = tok.tokenize(lower_case)
    words = tok.tokenize(letters_only)
    return (" ".join(words)).strip()

In [23]:
df_tweets['cleaned_tweets'] = df_tweets.tweet.progress_apply(tweet_cleaner)

In [24]:
df_tweets.head()

,geo,tweet,user_handle,location,cleaned_tweets
0,None,RT @DDNewslive: #PopeFrancis backs UN chief's ...,itsdubeyrishabh,"Gonda, Uttar Pradesh, India",RT PopeFrancis backs UN chief s call for a cea...
1,None,Memo to the @nytimes Maybe you should check wi...,drgrcevich,"Chagrin Falls, OH",Memo to the Maybe you should check with these ...
2,None,RT @GovNL: Physical (social) distancing means ...,eilanda23,,RT Physical social distancing means limiting y...
3,None,RT @DrBeniRusani: [BREAKING NEWS]\n\nThis doct...,gdwnsmnt_,,RT BREAKING NEWS This doctor identifies the mi...
4,None,"RT @RKSharma__: Divided by states, United by @...",Pparimal24,"Surat,Gujarat",RT Divided by states United by On the appeal o...


In [27]:
df_tweets['cleaned_tweets'] = df_tweets.cleaned_tweets.str.replace('RT', '')

In [28]:
df_tweets.head()

,geo,tweet,user_handle,location,cleaned_tweets
0,None,RT @DDNewslive: #PopeFrancis backs UN chief's ...,itsdubeyrishabh,"Gonda, Uttar Pradesh, India",PopeFrancis backs UN chief s call for a cease...
1,None,Memo to the @nytimes Maybe you should check wi...,drgrcevich,"Chagrin Falls, OH",Memo to the Maybe you should check with these ...
2,None,RT @GovNL: Physical (social) distancing means ...,eilanda23,,Physical social distancing means limiting you...
3,None,RT @DrBeniRusani: [BREAKING NEWS]\n\nThis doct...,gdwnsmnt_,,BREAKING NEWS This doctor identifies the mist...
4,None,"RT @RKSharma__: Divided by states, United by @...",Pparimal24,"Surat,Gujarat",Divided by states United by On the appeal of ...


In [29]:
df_tweets.to_csv('../Data/Tweets_scraped_2000_29th_CORONA_COVID.csv',index=False)